In [0]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15395907274086465292
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11280931226
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4152343868190128628
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0, compute capability: 3.7"
]


In [0]:
from keras.applications.densenet import DenseNet121

Using TensorFlow backend.


In [0]:
dense_net_conv = DenseNet121(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

In [0]:
# Freeze the layers except the last 4 layers
for layer in dense_net_conv.layers[:-4]:
    layer.trainable = False

# Check the trainable status of the individual layers
for layer in dense_net_conv.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7f022167d1d0> False
<keras.layers.convolutional.ZeroPadding2D object at 0x7f022167df60> False
<keras.layers.convolutional.Conv2D object at 0x7f022167dda0> False
<keras.layers.normalization.BatchNormalization object at 0x7f022161d160> False
<keras.layers.core.Activation object at 0x7f022161d630> False
<keras.layers.convolutional.ZeroPadding2D object at 0x7f0220583b70> False
<keras.layers.pooling.MaxPooling2D object at 0x7f02215e3da0> False
<keras.layers.normalization.BatchNormalization object at 0x7f022161de48> False
<keras.layers.core.Activation object at 0x7f0220511c88> False
<keras.layers.convolutional.Conv2D object at 0x7f02204c5da0> False
<keras.layers.normalization.BatchNormalization object at 0x7f02204e33c8> False
<keras.layers.core.Activation object at 0x7f0220444048> False
<keras.layers.convolutional.Conv2D object at 0x7f0220421978> False
<keras.layers.merge.Concatenate object at 0x7f02203f0240> False
<keras.layers.normalization.

In [0]:
from keras import models
from keras import layers
from keras import optimizers
 
# Create the model
model = models.Sequential()
 
# Add the densenet convolutional base model
model.add(dense_net_conv)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(200, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 1, 1, 1024)        7037504   
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 200)               205000    
Total params: 8,292,104
Trainable params: 1,293,512
Non-trainable params: 6,998,592
_________________________________________________________________


In [0]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.adamax(lr=1e-1),
              metrics=['acc'])

In [0]:
from keras.preprocessing.image import ImageDataGenerator

batch_size = 128
num_classes = 200
epochs = 5

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        "/home/ubuntu/DATA/ImageNet_Data/Train",
        target_size=(32, 32),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        '/home/ubuntu/DATA/ImageNet_Data/Validation',
        target_size=(32, 32),
        batch_size=batch_size,
        class_mode='categorical')

train_history = model.fit_generator(
        train_generator,
        steps_per_epoch=2000,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=200)    

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.
Epoch 1/5
 431/2000 [=====>........................] - ETA: 2:45 - loss: 16.0172 - acc: 0.0049

In [0]:
import os
# Save model and weights
save_dir = os.path.join(os.getcwd(), 'saved_models')
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_name = 'keras_Densenet_trained_model_TL.h5'
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

In [0]:
import matplotlib.pyplot as plt

acc = train_history.history['acc']
val_acc = train_history.history['val_acc']
loss = train_history.history['loss']
val_loss = train_history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
print(plt.show())

In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img
# Get the filenames from the generator
validation_dir = '/home/ubuntu/DATA/ImageNet_Data/Validation/'
fnames = validation_generator.filenames
 
# Get the ground truth from generator
ground_truth = validation_generator.classes
 
# Get the label to class mapping from the generator
label2index = validation_generator.class_indices
 
# Getting the mapping from class index to class label
idx2label = dict((v,k) for k,v in label2index.items())
 
# Get the predictions from the model using the generator
predictions = model.predict_generator(validation_generator, steps=validation_generator.samples/validation_generator.batch_size,verbose=1)
predicted_classes = np.argmax(predictions,axis=1)
 
errors = np.where(predicted_classes != ground_truth)[0]
print("No of errors = {}/{}".format(len(errors),validation_generator.samples))
 
# Show the errors
for i in range(len(errors)):
    pred_class = np.argmax(predictions[errors[i]])
    pred_label = idx2label[pred_class]
     
    title = 'Original label:{}, Prediction :{}, confidence : {:.3f}'.format(
        fnames[errors[i]].split('/')[0],
        pred_label,
        predictions[errors[i]][pred_class])
     
    original = load_img('{}/{}'.format(validation_dir,fnames[errors[i]]))
    plt.figure(figsize=[7,7])
    plt.axis('off')
    plt.title(title)
    plt.imshow(original)
    plt.show()